In [46]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [47]:
import warnings
warnings.filterwarnings('ignore')
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from copy import deepcopy
from xgboost import XGBClassifier


In [48]:
x_train = pd.read_csv('https://raw.githubusercontent.com/jjbl99/waterpump/master/xtrain_clean.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/jjbl99/waterpump/master/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/jjbl99/waterpump/master/xtest_clean.csv')

In [49]:
y_num = y_train.replace(['functional','non functional', 'functional needs repair'], [0,1,2]) 
x_test_id = x_test['id']
x_test = x_test.drop('id', axis = 1, inplace = True)

# Different Categorization techniques

In [50]:
# I think that would be logistical one
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c): 
            df[n] = pd.factorize(df[n])[0]
    return df
xtrain_log = train_cats(x_train)
xtest_log = train_cats(x_test)

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# mean target encoding
xcat_mean = deepcopy(x_train)
xtest_mean = deepcopy(x_test)

xcat_mean['target'] = deepcopy(y_num)['status_group']

for n,c in x_train.items():
        if is_string_dtype(c): 
            xcat_mean[n] = xcat_mean[n].map(xcat_mean.groupby(c)['target'].mean())
            xtest_mean[n] = xtest_mean[n].map(xcat_mean.groupby(c)['target'].mean()) 
            #DID IT WORK FOR TEST? How to stock the value?

xcat_mean = xcat_mean.drop('target', axis=1)

# Feature selection and PCA

In [ ]:
def FS_PCA(x_train, y_num):
    xtrain, xvalid, ytrain, yvalid = train_test_split(x_train, y_num, test_size=0.2)
    ytrain.drop('id', axis = 1, inplace = True)
    yvalid.drop('id', axis = 1, inplace = True)
    xtrain.drop('Unnamed: 0', axis = 1, inplace = True)
    xvalid.drop('Unnamed: 0', axis = 1, inplace = True)
    scaler = StandardScaler()
    feature_list = xtrain.columns

    xtrain_FS = xtrain
    xvalid_FS = xvalid

    xtrain_PCA = xtrain
    xvalid_PCA = xvalid

    xvalid_FS.drop(['date_recordedMonth','public_meeting','water_quality','quality_group','source_class','year_recorded','num_private'], axis = 1, inplace = True)
    xtrain_FS.drop(['year_recorded','source_class','date_recordedMonth','public_meeting','water_quality','quality_group','num_private'], axis = 1, inplace = True)

    pca = PCA(n_components= 15)
    pca.fit(xtrain) 
    PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,svd_solver='auto', tol=0.0, whiten=False)
    print(pca.explained_variance_ratio_)  

    xvalid_PCA = pca.transform(xvalid)
    xtrain_PCA = pca.transform(xtrain)
    
    return xtrain_FS, xvalid_FS, xtrain_PCA, xvalid_PCA, ytrain, yvalid

xtFS_log, xvFS_log, xtPCA_log, xvPCA_log, yt_log, yv_log = FS_PCA(xtrain_log, y_num)

xtFS_mean, xvFS_mean, xtPCA_mean, xvPCA_mean, yt_mean, yv_mean= FS_PCA(xcat_mean, y_num)

# Extremely Randomized Trees

In [51]:
def ext_RT (xtrain, xvalid, ytrain, yvalid):
    etc = ExtraTreesClassifier(n_estimators=300, 
                                    min_samples_split=8,
                                    max_features='sqrt',
                                    max_depth=95)
    etc.fit(xtrain, ytrain)
    predetc = etc.predict(xvalid)
    print('Training accuracy: ', accuracy_score(ytrain, etc.predict(xtrain)))
    print('Validation accuracy: ', accuracy_score(yvalid, predetc))
    return predetc

In [40]:
#feature selection: 
print('\nfor FS with mean: \n ')
ext_RT(xtFS_mean, xvFS_mean, yt_mean, yv_mean)

print('\nfor FS with log: \n')
ext_RT(xtFS_log, xvFS_log, yt_log, yv_log)

print('\nfor PCA with mean: \n ')
ext_RT(xtPCA_mean, xvPCA_mean, yt_mean, yv_mean)

print('\nfor PCA with log: \n ')
ext_RT(xtPCA_log, xvPCA_log, yt_log, yv_log)


for FS with mean: 
 
Training accuracy:  0.933733164983
Validation accuracy:  0.809175084175

for FS with log: 

Training accuracy:  0.93367003367
Validation accuracy:  0.804461279461

for PCA with mean: 
 
Training accuracy:  0.947874579125
Validation accuracy:  0.796212121212

for PCA with log: 
 
Training accuracy:  0.949221380471
Validation accuracy:  0.794865319865


# XGBoost

In [52]:
def XGB(xtrain, xvalid, ytrain, yvalid):
    xgb = XGBClassifier(max_depth=9, n_estimators=300,objective='multi:softmax') 
    xgb.fit(xtrain,ytrain)
    trainxgb = xgb.predict(xtrain)
    predxgb = xgb.predict(xvalid)
    print('Training accuracy: ', accuracy_score(ytrain, trainxgb))
    print('Validation accuracy: ', accuracy_score(yvalid, predxgb))
    return predxgb

In [42]:
#ytrain.drop('id', axis = 1, inplace = True)
#yvalid.drop('id', axis = 1, inplace = True)

In [43]:
#feature selection: 
print('\nfor FS with mean: \n ')
XGB(xtFS_mean, xvFS_mean, yt_mean, yv_mean)

print('\nfor FS with log: \n')
XGB(xtFS_log, xvFS_log, yt_log, yv_log)

#PCA
print('\nfor PCA with mean: \n ')
XGB(xtPCA_mean, xvPCA_mean, yt_mean, yv_mean)

print('\nfor PCA with log: \n ')
XGB(xtPCA_log, xvPCA_log, yt_log, yv_log)


for FS with mean: 
 
Training accuracy:  0.933038720539
Validation accuracy:  0.803451178451

for FS with log: 

Training accuracy:  0.936069023569
Validation accuracy:  0.804461279461

for PCA with mean: 
 
Training accuracy:  0.935101010101
Validation accuracy:  0.788383838384

for PCA with log: 
 
Training accuracy:  0.935206228956
Validation accuracy:  0.785774410774


# Working with the testing set

In [44]:
#We have xtest_mean, xtest_log

In [53]:
xtFS_log, xvFS_log, xtPCA_log, xvPCA_log, yt_log, yv_log = FS_PCA(xtest_log, y_num)

xtFS_mean, xvFS_mean, xtPCA_mean, xvPCA_mean, yt_mean, yv_mean= FS_PCA(xtest_mean, y_num)

xtrain_FS.drop(['year_recorded','source_class','date_recordedMonth','public_meeting','water_quality','quality_group','num_private'], axis = 1, inplace = True)

ValueError: Found input variables with inconsistent numbers of samples: [14850, 59400]

In [ ]:
prediction = ext_RT(xtFS_mean, xvFS_mean, yt_mean, yv_mean)

In [ ]:
prediction.head()